In [1]:
import re

# ================================
# 1️⃣ مرحلة التطبيع Normalize
# ================================
def normalize_arabic(word):
    # إزالة التشكيل
    word = re.sub(r'[\u064B-\u0652]', '', word)
    # توحيد أشكال الألف
    word = re.sub(r'[إأآا]', 'ا', word)
    # إزالة أي رموز أو أرقام
    word = re.sub(r'[^ء-ي]', '', word)
    return word


# ================================
# 2️⃣ الزوائد Prefixes & Suffixes
# ================================
PREFIXES = [
    "ال", "وال", "بال", "كال", "فال", "لل", "و", "ف", "ب", "ك", "ل", "س"
]

SUFFIXES = [
    "ه", "ها", "ك", "ي", "كم", "نا", "هما", "هم", "كن", "ين", "ون", 
    "ات", "ان", "تان", "تين", "ة", "ات", "وا", "ون", "ين"
]


def remove_affixes(word):
    # إزالة المقدمات
    for pre in sorted(PREFIXES, key=len, reverse=True):
        if word.startswith(pre) and len(word) - len(pre) >= 3:
            word = word[len(pre):]
            break
    # إزالة النهايات
    for suf in sorted(SUFFIXES, key=len, reverse=True):
        if word.endswith(suf) and len(word) - len(suf) >= 3:
            word = word[:-len(suf)]
            break
    return word


# ================================
# 3️⃣ أنماط الجذور (Patterns)
# ================================
PATTERNS = [
    "مفعول", "مفعلة", "فعالة", "افتعل", "استفعل", "تفاعل", "فعول", 
    "فعيل", "فاعل", "فعيلة", "انفعل", "افعل", "تفعّل", "فعّل", 
    "فعلى", "فعلة", "فعالة", "فعّال", "مفعل", "يفعل", "تفعلة"
]

# ================================
# 4️⃣ استخراج الجذر
# ================================
def extract_root(word):
    # نحاول مطابقة الكلمة مع الأنماط الصرفية
    for pattern in PATTERNS:
        regex = pattern.replace("ف", "(.?)").replace("ع", "(.?)").replace("ل", "(.?)")
        match = re.match(regex, word)
        if match:
            possible_root = "".join(match.groups())
            # تصفية الأحرف غير العربية
            possible_root = re.sub(r'[^ء-ي]', '', possible_root)
            if 3 <= len(possible_root) <= 4:
                return possible_root
    # fallback: إذا لم نجد نمطاً
    return word


# ================================
# 5️⃣ الدالة النهائية – Khoja Stemmer
# ================================
def khoja_stem(word, verbose=False):
    original = word
    word = normalize_arabic(word)
    no_affix = remove_affixes(word)
    root = extract_root(no_affix)

    if verbose:
        print(f"📘 الكلمة الأصلية: {original}")
        print(f"🔹 بعد التطبيع: {word}")
        print(f"🔹 بعد إزالة الزوائد: {no_affix}")
        print(f"✅ الجذر المستخرج: {root}")
        print("-" * 40)

    return root


# ================================
# 6️⃣ اختبار عملي متكامل
# ================================
text = """
المدارس يكتبون المعلمة مكتبة القراءة والمكتوبات فتعلم الطلاب الكتابة الصحيحة بالدفاتر.
"""

words = [w for w in text.split() if len(w) > 2]

print("🔹 KHOJA STEMMER – النسخة المحسّنة 🔹\n")
for w in words:
    khoja_stem(w, verbose=True)


🔹 KHOJA STEMMER – النسخة المحسّنة 🔹

📘 الكلمة الأصلية: المدارس
🔹 بعد التطبيع: المدارس
🔹 بعد إزالة الزوائد: مدارس
✅ الجذر المستخرج: دار
----------------------------------------
📘 الكلمة الأصلية: يكتبون
🔹 بعد التطبيع: يكتبون
🔹 بعد إزالة الزوائد: يكتب
✅ الجذر المستخرج: كتب
----------------------------------------
📘 الكلمة الأصلية: المعلمة
🔹 بعد التطبيع: المعلمة
🔹 بعد إزالة الزوائد: معلم
✅ الجذر المستخرج: علم
----------------------------------------
📘 الكلمة الأصلية: مكتبة
🔹 بعد التطبيع: مكتبة
🔹 بعد إزالة الزوائد: مكتب
✅ الجذر المستخرج: كتب
----------------------------------------
📘 الكلمة الأصلية: القراءة
🔹 بعد التطبيع: القراءة
🔹 بعد إزالة الزوائد: قراء
✅ الجذر المستخرج: قراء
----------------------------------------
📘 الكلمة الأصلية: والمكتوبات
🔹 بعد التطبيع: والمكتوبات
🔹 بعد إزالة الزوائد: مكتوب
✅ الجذر المستخرج: كتب
----------------------------------------
📘 الكلمة الأصلية: فتعلم
🔹 بعد التطبيع: فتعلم
🔹 بعد إزالة الزوائد: تعلم
✅ الجذر المستخرج: تعلم
--------------------------------------